<a href="https://colab.research.google.com/github/MatRitchie/Sunshine/blob/main/PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [9]:
from google.colab import userdata, files
import os
import pandas as pd
import numpy as np

Acces github

In [10]:
os.environ['GITHUB_TOKEN'] = userdata.get('GITHUB_TOKEN')
%cd /content
!git clone https://$GITHUB_TOKEN@github.com/MatRitchie/Sunshine.git

%cd /content/Sunshine
!git pull https://$GITHUB_TOKEN@github.com/MatRitchie/Sunshine.git main

/content
fatal: destination path 'Sunshine' already exists and is not an empty directory.
/content/Sunshine
From https://github.com/MatRitchie/Sunshine
 * branch            main       -> FETCH_HEAD
Already up to date.


In [11]:
FILES = {
    "evolved": "incubator_project_graduated_from_graduation.csv",
    "retired": "incubator_project_metrics_graduated_retired.csv",
    "bypassed": "non_incubator_project_metrics.csv"
}
OUTPUT_FILE = "combined_project_status.csv"

LABELS = [
    'project', 'COM-1', 'COM-3', 'POP-4', 'STA-1', 'STA-2', 'STA-4', 'STA-5',
    'STA-6', 'STA-7', 'STA-10', 'STA-11', 'STA-12', 'TEC-1', 'TEC-2.1',
    'TEC-2.2', 'TEC-2.3', 'TEC-2.4', 'TEC-4', 'TEC-5', 'SWQ-1', 'SWQ-4.1',
    'SWQ-4.2', 'SWQ-4.3', 'SWQ-4.4', 'SWQ-4.5', 'SWQ-4.6', 'SWQ-4.7',
    'init', 'end', 'frequency (weeks)', 'project_url'
]

##Load and process the files

In [ ]:
dfs = {}
for status, filepath in FILES.items():
    try:
        print(f"\nProcessing file for status: {status}, path: {filepath}") 
        df = pd.read_csv(filepath)
        print(f"  Number of DATA ROWS read by pandas for '{status}': {len(df)}") 

        if status == "bypassed":
            with open(filepath, 'r') as f_temp:
                num_lines_in_file = sum(1 for line_in_f in f_temp)
            print(f"  Total lines in the actual file '{filepath}': {num_lines_in_file}")
            print(f"  Content of the first few rows read by pandas for '{status}':\n{df.head()}")


        df.columns = LABELS[:min(len(df.columns), len(LABELS))]

        df.rename(columns={
            'COM-1'   : 'COM-1',
            'COM-3'   : 'COM-2',
            'POP-4'   : 'POP-1',
            'STA-1'   : 'STA-1',
            'STA-2'   : 'STA-2',
            'STA-4'   : 'STA-3',
            'STA-5'   : 'STA-4',
            'STA-6'   : 'STA-5',
            'STA-7'   : 'STA-6',
            'STA-10'  : 'STA-7',
            'STA-11'  : 'STA-8',
            'STA-12'  : 'STA-9',
            'TEC-1'   : 'TEC-1',
            'TEC-2.4' : 'TEC-2',
            'TEC-4'   : 'TEC-3',
            'TEC-5'   : 'TEC-4',
            'SWQ-1'   : 'SWQ-1',
            'SWQ-4.1' : 'SWQ-2.1',
            'SWQ-4.2' : 'SWQ-2.2',
            'SWQ-4.3' : 'SWQ-2.3',
            'SWQ-4.4' : 'SWQ-2.4',
            'SWQ-4.5' : 'SWQ-2.5',
            'SWQ-4.6' : 'SWQ-2.6',
            'SWQ-4.7' : 'SWQ-2.7',
        }, inplace=True)

        if 'TEC-2.2' in df.columns:
          df.drop(columns=['TEC-2.2'], inplace=True)

        if 'frequency (weeks)' in df.columns:
          df.drop(columns=['frequency (weeks)'], inplace=True)

        if 'STA-6' in df.columns:
          df['STA-6'] = df['STA-6'].apply(lambda x: 1 if str(x).lower() == 'true' else (0 if str(x).lower() == 'false' else x))

        # Calculate project age from STA-1 if its there
        if 'STA-1' in df.columns:
            df['STA-1'] = (2025 - pd.to_datetime(df['STA-1'], errors='coerce').dt.year).fillna(-1).astype(int)

        df['status'] = status
        dfs[status] = df
        print(f"  Number of rows for '{status}' after processing in loop: {len(df)}") 
    except Exception as e:
        print(f"Error processing {filepath}: {e}")


Processing file for status: evolved, path: incubator_project_graduated_from_graduation.csv
  Number of DATA ROWS read by pandas for 'evolved': 222
  Number of rows for 'evolved' after processing in loop: 222

Processing file for status: retired, path: incubator_project_metrics_graduated_retired.csv
  Number of DATA ROWS read by pandas for 'retired': 282
  Number of rows for 'retired' after processing in loop: 282

Processing file for status: bypassed, path: non_incubator_project_metrics.csv
  Number of DATA ROWS read by pandas for 'bypassed': 114
  Total lines in the actual file 'non_incubator_project_metrics.csv': 115
  Content of the first few rows read by pandas for 'bypassed':
    ant-antlibs-dotnet      Issues missing Issues missing.1     17  \
0                  ant      Issues missing   Issues missing   1256   
1  ant-antlibs-antunit      Issues missing   Issues missing     30   
2                arrow   6105807.218980744           172658  31660   
3                 avro   6462

# Set project status based on presence in evolved list

In [13]:
if 'evolved' in dfs and 'retired' in dfs:
    evolved_projects = set(dfs['evolved']['project'])
    dfs['retired']['status'] = dfs['retired']['project'].apply(
        lambda proj: 'graduated' if proj in evolved_projects else 'retired'
    )

In [ ]:
df_combined = pd.concat([df for df in dfs.values() if df is not None], ignore_index=True)

print("--- Counts per status in df_combined (raw, after concat, before any filtering in this cell) ---")
print(df_combined['status'].value_counts())
print("Is 'fluo-yarn' present in df_combined at this stage?", 'fluo-yarn' in df_combined['project'].values)
if 'fluo-yarn' in df_combined['project'].values:
    print("Status of 'fluo-yarn' in df_combined:", df_combined[df_combined['project'] == 'fluo-yarn']['status'].values)
print("Shape of df_combined:", df_combined.shape)
print("--- End of initial df_combined stats ---\n")

df_base_for_row_removal_stats = df_combined.dropna(axis=1, how='all').copy()

print("--- Counts per status in df_base_for_row_removal_stats (after column dropna, before fluo-yarn/row removal) ---")
print(df_base_for_row_removal_stats['status'].value_counts())
print("Is 'fluo-yarn' present in df_base_for_row_removal_stats?", 'fluo-yarn' in df_base_for_row_removal_stats['project'].values)
if 'fluo-yarn' in df_base_for_row_removal_stats['project'].values:
    print("Status of 'fluo-yarn' in df_base_for_row_removal_stats:", df_base_for_row_removal_stats[df_base_for_row_removal_stats['project'] == 'fluo-yarn']['status'].values)
print("Shape of df_base_for_row_removal_stats:", df_base_for_row_removal_stats.shape)
print("--- End of df_base_for_row_removal_stats stats ---\n")

df_for_cleaning = df_base_for_row_removal_stats.copy()

# remove bad project fluo-yarn
if 'fluo-yarn' in df_for_cleaning['project'].values:
    print("Removing 'fluo-yarn'...")
    df_for_cleaning = df_for_cleaning[df_for_cleaning['project'] != 'fluo-yarn']
else:
    print("'fluo-yarn' not found in df_for_cleaning before explicit removal step.")


# remove rows with strings that say "missing"
df_for_cleaning = df_for_cleaning[~df_for_cleaning.astype(str).apply(lambda x: x.str.contains('missing', case=False, na=False)).any(axis=1)]

df_clean = df_for_cleaning.dropna()

print("--- Counts per status in final df_clean (after all cleaning) ---")
print(df_clean['status'].value_counts())
print("Is 'fluo-yarn' present in df_clean?", 'fluo-yarn' in df_clean['project'].values)
print("Shape of df_clean:", df_clean.shape)
print("--- End of df_clean stats ---\n")

try:
    df_clean.to_csv(OUTPUT_FILE, index=False)
except Exception as e:
    print(f"Error saving output: {e}")

--- Counts per status in df_combined (raw, after concat, before any filtering in this cell) ---
status
evolved      222
graduated    222
bypassed     114
retired       60
Name: count, dtype: int64
Is 'fluo-yarn' present in df_combined at this stage? True
Status of 'fluo-yarn' in df_combined: ['bypassed']
Shape of df_combined: (618, 31)
--- End of initial df_combined stats ---

--- Counts per status in df_base_for_row_removal_stats (after column dropna, before fluo-yarn/row removal) ---
status
evolved      222
graduated    222
bypassed     114
retired       60
Name: count, dtype: int64
Is 'fluo-yarn' present in df_base_for_row_removal_stats? True
Status of 'fluo-yarn' in df_base_for_row_removal_stats: ['bypassed']
Shape of df_base_for_row_removal_stats: (618, 29)
--- End of df_base_for_row_removal_stats stats ---

Removing 'fluo-yarn'...
--- Counts per status in final df_clean (after all cleaning) ---
status
evolved      189
graduated    189
bypassed      99
retired       47
Name: count

#Missing data removal analysis

In [ ]:
projects_in_baseline = set(df_base_for_row_removal_stats['project'])
projects_in_clean = set(df_clean['project'])

removed_project_names = projects_in_baseline - projects_in_clean
df_removed_projects = df_base_for_row_removal_stats[df_base_for_row_removal_stats['project'].isin(removed_project_names)]
counts_of_removed_projects_by_status = df_removed_projects['status'].value_counts()

initial_total_counts_by_status = df_base_for_row_removal_stats['status'].value_counts()
all_statuses = sorted(list(df_base_for_row_removal_stats['status'].unique()))

print("\nStatistics on projects removed (includes 'fluo-yarn', rows with 'missing' text, or NaN data):")
total_projects_in_baseline = len(projects_in_baseline)
total_rows_in_baseline = len(df_base_for_row_removal_stats)
total_projects_removed = len(removed_project_names) 
total_rows_removed = len(df_removed_projects)
total_projects_remaining = len(projects_in_clean)
total_rows_remaining = len(df_clean)

print(f"\nOverall Project Row Counts:")
print(f"  Total project entries in baseline (before specific removals): {total_rows_in_baseline}")
print(f"  Total project entries removed: {total_rows_removed}")
print(f"  Total project entries remaining in df_clean: {total_rows_remaining}")

print("\nBreakdown by project status (based on project entries/rows):")
for status in all_statuses:
    initial_row_count = initial_total_counts_by_status.get(status, 0)
    removed_row_count = counts_of_removed_projects_by_status.get(status, 0)

    print(f"\nStatus: {status}")
    print(f"  Number of project entries initially for this status: {initial_row_count}")
    print(f"  Number of project entries removed for this status: {removed_row_count}")

    if initial_row_count > 0:
        remaining_row_count = initial_row_count - removed_row_count
        proportion_left = remaining_row_count / initial_row_count
        print(f"  Number of project entries remaining for this status: {remaining_row_count}")
        print(f"  Proportion of project entries left for this status: {proportion_left:.2%}")
    elif removed_row_count > 0:
        print(f"  Warning: {removed_row_count} project entries removed, but initial count for this status was 0 in the baseline. This might indicate an issue or a project changing status before removal consideration.")
    else:
        print(f"  No project entries found for this status in the baseline or among removed.")

print("\n--- End of Missing Data Removal Analysis ---")

--- Missing Data Removal Analysis ---

Statistics on projects removed (includes 'fluo-yarn', rows with 'missing' text, or NaN data):

Overall Project Row Counts:
  Total project entries in baseline (before specific removals): 618
  Total project entries removed: 94
  Total project entries remaining in df_clean: 524
  Note: 'fluo-yarn' was among the removed projects. Its status at time of removal consideration was: bypassed

Breakdown by project status (based on project entries/rows):

Status: bypassed
  Number of project entries initially for this status: 114
  Number of project entries removed for this status: 15
  Number of project entries remaining for this status: 99
  Proportion of project entries left for this status: 86.84%

Status: evolved
  Number of project entries initially for this status: 222
  Number of project entries removed for this status: 33
  Number of project entries remaining for this status: 189
  Proportion of project entries left for this status: 85.14%

Status

#Pushing processed data

In [ ]:
# Stage the new files for Git
!git add {OUTPUT_FILE}
!git commit -m "Add preprocessed data for combined files with removed missing data"

!git config --global user.email "auto@example.com"
!git config --global user.name "Auto"   

!git push https://$GITHUB_TOKEN@github.com/MatRitchie/Sunshine.git main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
